---
title: Use automated ML in ML pipelines 
titleSuffix: Azure Machine Learning
description: The AutoMLStep allows you to use automated machine learning in your pipelines.
services: machine-learning
ms.service: machine-learning
ms.subservice: core
ms.topic: conceptual
ms.author: laobri
author: lobrien
manager: cgronlun
ms.date: 04/28/2020

---

# Use automated ML in an Azure Machine Learning pipeline in Python
[!INCLUDE [applies-to-skus](../../includes/aml-applies-to-basic-enterprise-sku.md)]

Azure Machine Learning's automated ML capability helps you discover high-performing models without you reimplementing every possible approach. Combined with Azure Machine Learning pipelines, you can create deployable workflows that can quickly discover the algorithm that works best for your data. This article will show you how to efficiently join a data preparation step to an automated ML step. Automated ML can quickly discover the algorithm that works best for your data, while putting you on the road to MLOps and model lifecycle operationalization with pipelines.

## Prerequisites

* An Azure subscription. If you don't have an Azure subscription, create a free account before you begin. Try the [free or paid version of Azure Machine Learning](https://aka.ms/AMLFree) today.

* An Azure Machine Learning workspace. See [Create an Azure Machine Learning workspace](how-to-manage-workspace.md).  

* Basic familiarity with Azure's [automated machine learning](concept-automated-ml.md) and [machine learning pipelines](concept-ml-pipelines.md) facilities and SDK.

## Review automated ML's central classes

Automated ML in a pipeline is represented by an `AutoMLStep` object. The `AutoMLStep` class is a subclass of `PipelineStep`. A graph of `PipelineStep` objects defines a `Pipeline`.

There are several subclasses of `PipelineStep`. In addition to the `AutoMLStep`, this article will show a `PythonScriptStep` for data preparation and another for registering the model.

The preferred way to initially move data _into_ an ML pipeline is with `Dataset` objects. To move data _between_ steps, the preferred way is with `PipelineData` objects. To be used with `AutoMLStep`, the `PipelineData` object must be transformed into a `PipelineOutputTabularDataset` object. For more information, see [Input and output data from ML pipelines](how-to-move-data-in-out-of-pipelines.md).

The `AutoMLStep` is configured via an `AutoMLConfig` object. `AutoMLConfig` is a flexible class, as discussed in [Configure automated ML experiments in Python](https://docs.microsoft.com/azure/machine-learning/how-to-configure-auto-train#configure-your-experiment-settings). 

A `Pipeline` runs in an `Experiment`. The pipeline `Run` has, for each step, a child `StepRun`. The outputs of the automated ML `StepRun` are the training metrics and highest-performing model.

To make things concrete, this article creates a simple pipeline for a classification task. The task is predicting Titanic survival, but we won't be discussing the data or task except in passing.

## Get started

### Retrieve initial dataset

Often, an ML workflow starts with pre-existing baseline data. This is a good scenario for a registered dataset. Datasets are visible across the workspace, support versioning, and can be interactively explored. There are many ways to create and populate a dataset, as discussed in [Create Azure Machine Learning datasets](how-to-create-register-datasets.md). Since we'll be using the Python SDK to create our pipeline, use the SDK to download baseline data and register it with the name 'titanic_ds'.

In [2]:
from azureml.core import Workspace, Dataset
from azureml.core.authentication import InteractiveLoginAuthentication
import os

ws = Workspace.from_config(auth=InteractiveLoginAuthentication(tenant_id=os.environ["AML_TENANT_ID"]))
#ws = Workspace.from_config()
if not 'titanic_ds' in ws.datasets.keys() :
    # create a TabularDataset from Titanic training data
    web_paths = ['https://dprepdata.blob.core.windows.net/demo/Titanic.csv',
                 'https://dprepdata.blob.core.windows.net/demo/Titanic2.csv']
    titanic_ds = Dataset.Tabular.from_delimited_files(path=web_paths)

    titanic_ds.register(workspace = ws,
                                     name = 'titanic_ds',
                                     description = 'Titanic baseline data',
                                     create_new_version = True)

titanic_ds = Dataset.get_by_name(ws, 'titanic_ds')

The code first logs in to the Azure Machine Learning workspace defined in **config.json** (for an explanation, see [Tutorial: Get started creating your first ML experiment with the Python SDK](tutorial-1st-experiment-sdk-setup.md)). If there isn't already a dataset named `'titanic_ds'` registered, then it creates one. The code downloads CSV data from the Web, uses them to instantiate a `TabularDataset` and then registers the dataset with the workspace. Finally, the function `Dataset.get_by_name()` assigns the `Dataset` to `titanic_ds`. 

### Configure your storage and compute target

Additional resources that the pipeline will need are storage and, generally, Azure Machine Learning compute resources.

In [10]:
from azureml.core import Datastore
from azureml.core.compute import AmlCompute, ComputeTarget

datastore = ws.get_default_datastore()

compute_name = 'cpu-compute3'
if not compute_name in ws.compute_targets :
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                                min_nodes=0,
                                                                max_nodes=1)
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

    compute_target.wait_for_completion(
        show_output=True, min_node_count=None, timeout_in_minutes=20)

    # Show the result
    print(compute_target.get_status().serialize())

compute_target = ws.compute_targets[compute_name]

The intermediate data between the data preparation and the automated ML step can be stored in the workspace's default datastore, so we don't need to do more than call `get_default_datastore()` on the `Workspace` object. 

After that, the code checks if the AML compute target `'cpu-cluster'` already exists. If not, we specify that we want a small CPU-based compute target. If you plan to use automated ML's deep learning features (for instance, text featurization with DNN support) you should choose a compute with strong GPU support, as described in [GPU optimized virtual machine sizes](https://docs.microsoft.com/azure/virtual-machines/sizes-gpu). 

The code blocks until the target is provisioned and then prints some details of the just-created compute target. Finally, the named compute target is retrieved from the workspace and assigned to `compute_target`. 

### Configure the training run

The next step is making sure that the remote training run has all the dependencies that are required by the training steps. Dependencies and the runtime context are set by creating and configuring a `RunConfiguration` object.

In [11]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

aml_run_config = RunConfiguration()
# Use just-specified compute target ("cpu-cluster")
aml_run_config.target = compute_target
aml_run_config.environment.python.user_managed_dependencies = False

# Add some packages relied on by data prep step
aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
    conda_packages=['pandas','scikit-learn'], 
    pip_packages=['azureml-sdk[automl,explain]', 'azureml-dataprep[fuse,pandas]'], 
    pin_sdk_version=False)

## Prepare data for automated machine learning

### Write the data preparation code

The baseline Titanic dataset consists of mixed numerical and text data, with some values missing. To prepare it for automated machine learning, the data preparation pipeline step will:

- Fill missing data with either random data or a category corresponding to "Unknown"
- Transform categorical data to integers
- Drop columns that we don't intend to use
- Split the data into training and testing sets
- Write the transformed data to the `PipelineData` output paths

In [12]:
%%writefile dataprep.py
from azureml.core import Run

import pandas as pd 
import numpy as np 
import pyarrow as pa
import pyarrow.parquet as pq
import argparse

RANDOM_SEED=42

def prepare_age(df):
    # Fill in missing Age values from distribution of present Age values 
    mean = df["Age"].mean()
    std = df["Age"].std()
    is_null = df["Age"].isnull().sum()
    # compute enough (== is_null().sum()) random numbers between the mean, std
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)
    # fill NaN values in Age column with random values generated
    age_slice = df["Age"].copy()
    age_slice[np.isnan(age_slice)] = rand_age
    df["Age"] = age_slice
    df["Age"] = df["Age"].astype(int)
    
    # Quantize age into 5 classes
    df['Age_Group'] = pd.qcut(df['Age'],5, labels=False)
    df.drop(['Age'], axis=1, inplace=True)
    return df

def prepare_fare(df):
    df['Fare'].fillna(0, inplace=True)
    df['Fare_Group'] = pd.qcut(df['Fare'],5,labels=False)
    df.drop(['Fare'], axis=1, inplace=True)
    return df 

def prepare_genders(df):
    genders = {"male": 0, "female": 1, "unknown": 2}
    df['Sex'] = df['Sex'].map(genders)
    df['Sex'].fillna(2, inplace=True)
    df['Sex'] = df['Sex'].astype(int)
    return df

def prepare_embarked(df):
    df['Embarked'].replace('', 'U', inplace=True)
    df['Embarked'].fillna('U', inplace=True)
    ports = {"S": 0, "C": 1, "Q": 2, "U": 3}
    df['Embarked'] = df['Embarked'].map(ports)
    return df
    
parser = argparse.ArgumentParser()
parser.add_argument('--output_path', dest='output_path', required=True)
args = parser.parse_args()
    
titanic_ds = Run.get_context().input_datasets['titanic_ds']
df = titanic_ds.to_pandas_dataframe().drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
df = prepare_embarked(prepare_genders(prepare_fare(prepare_age(df))))

os.makedirs(os.path.dirname(args.output_path), exist_ok=True)
pq.write_table(pa.Table.from_pandas(df), args.output_path)

print(f"Wrote test to {args.output_path} and train to {args.output_path}")

Overwriting dataprep.py


The above code snippet is a complete, but minimal, example of data preparation for the Titanic data. The snippet starts with a Jupyter "magic command" to output the code to a file. If you aren't using a Jupyter notebook, remove that line and create the file manually.

The various `prepare_` functions in the above snippet modify the relevant column in the input dataset. These functions work on the data once it has been changed into a Pandas `DataFrame` object. In each case, missing data is either filled with representative random data or categorical data indicating "Unknown." Text-based categorical data is mapped to integers. No-longer-needed columns are overwritten or dropped. 

After the code defines the data preparation functions, the code parses the input argument, which is the path to which we want to write our data. (These values will be determined by `PipelineData` objects that will be discussed in the next step.) The code retrieves the registered `'titanic_cs'` `Dataset`, converts it to a Pandas `DataFrame`, and calls the various data preparation functions. 

Since the `output_path` is fully qualified, the function `os.makedirs()` is used to prepare the directory structure. At this point, you could use `DataFrame.to_csv()` to write the output data, but Parquet files are  more efficient. This efficiency would probably be irrelevant with such a small dataset, but using the **PyArrow** package's `from_pandas()` and `write_table()` functions are only a few more keystrokes than `to_csv()`.

Parquet files are natively supported by the automated ML step discussed below, so no special processing is required to consume them. 

### Write the data preparation pipeline step (`PythonScriptStep`)

The data preparation code described above must be associated with a `PythonScripStep` object to be used with a pipeline. The path to which the Parquet data-preparation output is written is generated by a `PipelineData` object. The resources prepared earlier, such as the `ComputeTarget`, the `RunConfig`, and the `'titanic_ds' Dataset` are used to complete the specification.

In [13]:
from azureml.pipeline.core import PipelineData
from azureml.pipeline.steps import PythonScriptStep

prepped_data_path = PipelineData("titanic_train", datastore).as_dataset()
prepped_data_path = PipelineData("titanic_train", datastore).as_dataset()

dataprep_step = PythonScriptStep(
    name="dataprep", 
    script_name="dataprep.py", 
    compute_target=compute_target, 
    runconfig=aml_run_config,
    arguments=["--output_path", prepped_data_path],
    inputs=[titanic_ds.as_named_input("titanic_ds")],
    outputs=[prepped_data_path],
    allow_reuse=True
)

The `prepped_data_path` object is of type `PipelineOutputFileDataset`. Notice that it's specified in both the `arguments` and `outputs` arguments. If you review the previous step, you'll see that within the data preparation code, the value of the argument `'--output_path'` is the file path to which the Parquet file was written. 

## Train with AutoMLStep

Configuring an automated ML pipeline step is done with the `AutoMLConfig` class. This flexible class is described in [Configure automated ML experiments in Python](https://docs.microsoft.com/azure/machine-learning/how-to-configure-auto-train). Data input and output are the only aspects of configuration that require special attention in an ML pipeline. Input and output for `AutoMLConfig` in pipelines is discussed in detail below. Beyond data, an advantage of ML pipelines is the ability to use different compute targets for different steps. You might choose to use a more powerful `ComputeTarget` only for the automated ML process. Doing so is as straightforward as assigning a more powerful `RunConfiguration` to the `AutoMLConfig` object's `run_configuration` parameter.

### Send data to `AutoMLStep`

In an ML pipeline, the input data must be a `Dataset` object. The highest-performing way is to provide the input data in the form of `PipelineOutputTabularDataset` objects. You create an object of that type with the `parse_parquet_files()` or `parse_delimited_files()` on a `PipelineOutputFileDataset`, such as the `prepped_data_path` object.

In [14]:
# type(prepped_data_path) == PipelineOutputFileDataset
# type(prepped_data) == PipelineOutputTabularDataset
prepped_data = prepped_data_path.parse_parquet_files(file_extension=None)

The snippet above creates a high-performing `PipelineOutputTabularDataset` from the `PipelineOutputFileDataset` output of the data preparation step.

Another option is to use `Dataset` objects registered in the workspace:

In [16]:
#prepped_data = Dataset.get_by_name(ws, 'Data_prepared')

Comparing the two techniques:

| Technique |  | 
|-|-|
|`PipelineOutputTabularDataset`| Higher performance | 
|| Natural route from `PipelineData` | 
|| Data isn't persisted after pipeline run |
|| [Notebook showing `PipelineOutputTabularDataset` technique](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/machine-learning-pipelines/nyc-taxi-data-regression-model-building/nyc-taxi-data-regression-model-building.ipynb) |
| Registered `Dataset` | Lower performance |
| | Can be generated in many ways | 
| | Data persists and is visible throughout workspace |
| | [Notebook showing registered `Dataset` technique](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/automated-machine-learning/continuous-retraining/auto-ml-continuous-retraining.ipynb)

### Specify automated ML outputs

The outputs of the `AutoMLStep` are the final metric scores of the higher-performing model and that model itself. To use these outputs in further pipeline steps, prepare `PipelineData` objects to receive them.

In [17]:
from azureml.pipeline.core import TrainingOutput

metrics_data = PipelineData(name='metrics_data',
                           datastore=datastore,
                           pipeline_output_name='metrics_output',
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='best_model_data',
                           datastore=datastore,
                           pipeline_output_name='model_output',
                           training_output=TrainingOutput(type='Model'))

The snippet above creates the two `PipelineData` objects for the metrics and model output. Each is named, assigned to the default datastore retrieved earlier, and associated with the particular `type` of `TrainingOutput` from the `AutoMLStep`. 

### Configure and create the automated ML pipeline step

Once the inputs and outputs are defined, it's time to create the `AutoMLConfig` and `AutoMLStep`. The details of the configuration will depend on your task, as described in [Configure automated ML experiments in Python](https://docs.microsoft.com/azure/machine-learning/how-to-configure-auto-train]). For the Titanic survival classification task, the following snippet demonstrates a simple configuration.

In [18]:
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep

# Change iterations to a reasonable number (50) to get better accuracy
automl_settings = {
    "iteration_timeout_minutes" : 10,
    "iterations" : 2,
    "experiment_timeout_hours" : 0.25,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(task = 'classification',
                             path = '.',
                             debug_log = 'automated_ml_errors.log',
                             compute_target = compute_target,
                             run_configuration = aml_run_config,
                             featurization = 'auto',
                             training_data = prepped_data,
                             label_column_name = 'Survived',
                             **automl_settings)

train_step = AutoMLStep(name='AutoML_Classification',
    automl_config=automl_config,
    passthru_automl_config=False,
    outputs=[metrics_data,model_data],
    allow_reuse=True)

The snippet shows an idiom commonly used with `AutoMLConfig`. Arguments that are more fluid (hyperparameter-ish) are specified in a separate dictionary while the values less likely to change are specified directly in the `AutoMLConfig` constructor. In this case, the `automl_settings` specify a brief run: the run will stop after only 2 iterations or 15 minutes, whichever comes first.

The `automl_settings` dictionary is passed to the `AutoMLConfig` constructor as kwargs. The other parameters aren't complex:

- `task` is set to `classification` for this example. Other valid values are `regression` and `forecasting`
- `path` and `debug_log` describe the path to the project and a local file to which debug information will be written 
- `compute_target` is the previously defined `compute_target` that, in this example, is an inexpensive CPU-based machine. If you're using AutoML's Deep Learning facilities, you would want to change the compute target to be GPU-based
- `featurization` is set to `auto`. More details can be found in the [Data Featurization](https://docs.microsoft.com/azure/machine-learning/how-to-configure-auto-train#data-featurization) section of the automated ML configuration document 
- `training_data` is set to the `PipelineOutputTabularDataset` objects made from the outputs of the data preparation step 
- `label_column_name` indicates which column we are interested in predicting 

The `AutoMLStep` itself takes the `AutoMLConfig` and has, as outputs, the `PipelineData` objects created to hold the metrics and model data. 

>[!Important]
> You must set `passthru_automl_config` to `False` if your `AutoMLStep` is using `PipelineOutputTabularDataset` objects for input.

In this example, the automated ML process will perform cross-validations on the `training_data`. You can control the number of cross-validations with the `n_cross_validations` argument. If you've already split your training data as part of your data preparation steps, you can set `validation_data` to its own `Dataset`.

You might occasionally see the use `X` for data features and `y` for data labels. This technique is deprecated and you should use `training_data` for input. 

## Register the model generated by automated ML 

The last step in a basic ML pipeline is registering the created model. By adding the model to the workspace's model registry, it will be available in the portal and can be versioned. To register the model, write another `PythonScriptStep` that takes the `model_data` output of the `AutoMLStep`.

### Write the code to register the model

A model is registered in a `Workspace`. You're probably familiar with using `Workspace.from_config()` to log on to your workspace on your local machine, but there's another way to get the workspace from within a running ML pipeline. The `Run.get_context()` retrieves the active `Run`. This `run` object provides access to many important objects, including the `Workspace` used here.

In [19]:
%%writefile register_model.py
from azureml.core.model import Model, Dataset
from azureml.core.run import Run, _OfflineRun
from azureml.core import Workspace
import argparse

parser = argparse.ArgumentParser()
parser.add_argument("--model_name", required=True)
parser.add_argument("--model_path", required=True)
args = parser.parse_args()

print(f"model_name : {args.model_name}")
print(f"model_path: {args.model_path}")

run = Run.get_context()
ws = Workspace.from_config() if type(run) == _OfflineRun else run.experiment.workspace

model = Model.register(workspace=ws,
                       model_path=args.model_path,
                       model_name=args.model_name)

print("Registered version {0} of model {1}".format(model.version, model.name))

Overwriting register_model.py


### Write the PythonScriptStep code

The model-registering `PythonScriptStep` uses a `PipelineParameter` for one of its arguments. Pipeline parameters are arguments to pipelines that can be easily set at run-submission time. Once declared, they're passed as normal arguments.

In [20]:

from azureml.pipeline.core.graph import PipelineParameter

# The model name with which to register the trained model in the workspace.
model_name = PipelineParameter("model_name", default_value="TitanicSurvivalInitial")

register_step = PythonScriptStep(script_name="register_model.py",
                                       name="register_model",
                                       allow_reuse=False,
                                       arguments=["--model_name", model_name, "--model_path", model_data],
                                       inputs=[model_data],
                                       compute_target=compute_target,
                                       runconfig=aml_run_config)

## Create and run your automated ML pipeline

Creating and running a pipeline that contains an `AutoMLStep` is no different than a normal pipeline.

In [ ]:
from azureml.pipeline.core import Pipeline
from azureml.core import Experiment

pipeline = Pipeline(ws, [dataprep_step, train_step, register_step])

experiment = Experiment(workspace=ws, name='titanic_automl')

run = experiment.submit(pipeline, show_output=True)
run.wait_for_completion()

Created step dataprep [535fdcb1][6acdb23b-bba1-442d-bc41-99a37d43a304], (This step will run and generate new outputs)Created step AutoML_Classification [c29ef81e][371e2f74-f181-495f-8f12-2cee917ba054], (This step will run and generate new outputs)

Created step register_model [35832e40][aaee60ca-1f20-4d11-8588-1d04101ed48f], (This step will run and generate new outputs)
Submitted PipelineRun 3b6e6d42-a2bb-41bb-9a44-aba546e3acb4
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/titanic_automl/runs/3b6e6d42-a2bb-41bb-9a44-aba546e3acb4?wsid=/subscriptions/65a1016d-0f67-45d2-b838-b8f373d6d52e/resourcegroups/laobri-ml/workspaces/pipelines
PipelineRunId: 3b6e6d42-a2bb-41bb-9a44-aba546e3acb4
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/titanic_automl/runs/3b6e6d42-a2bb-41bb-9a44-aba546e3acb4?wsid=/subscriptions/65a1016d-0f67-45d2-b838-b8f373d6d52e/resourcegroups/laobri-ml/workspaces/pipelines
PipelineRun Status: NotStarted
PipelineRun Status: R


mkl-2019.4           | 204.1 MB  | ########4  |  85% 
mkl-2019.4           | 204.1 MB  | ########4  |  85% 
mkl-2019.4           | 204.1 MB  | ########4  |  85% 
mkl-2019.4           | 204.1 MB  | ########4  |  85% 
mkl-2019.4           | 204.1 MB  | ########5  |  85% 
mkl-2019.4           | 204.1 MB  | ########5  |  85% 
mkl-2019.4           | 204.1 MB  | ########5  |  85% 
mkl-2019.4           | 204.1 MB  | ########5  |  85% 
mkl-2019.4           | 204.1 MB  | ########5  |  85% 
mkl-2019.4           | 204.1 MB  | ########5  |  85% 
mkl-2019.4           | 204.1 MB  | ########5  |  85% 
mkl-2019.4           | 204.1 MB  | ########5  |  86% 
mkl-2019.4           | 204.1 MB  | ########5  |  86% 
mkl-2019.4           | 204.1 MB  | ########5  |  86% 
mkl-2019.4           | 204.1 MB  | ########5  |  86% 
mkl-2019.4           | 204.1 MB  | ########5  |  86% 
mkl-2019.4           | 204.1 MB  | ########5  |  86% 
mkl-2019.4           | 204.1 MB  | ########5  |  86% 
mkl-2019.4           | 204.


mkl-2019.4           | 204.1 MB  | #########6 |  96% 
mkl-2019.4           | 204.1 MB  | #########6 |  97% 
mkl-2019.4           | 204.1 MB  | #########6 |  97% 
mkl-2019.4           | 204.1 MB  | #########6 |  97% 
mkl-2019.4           | 204.1 MB  | #########6 |  97% 
mkl-2019.4           | 204.1 MB  | #########6 |  97% 
mkl-2019.4           | 204.1 MB  | #########6 |  97% 
mkl-2019.4           | 204.1 MB  | #########6 |  97% 
mkl-2019.4           | 204.1 MB  | #########6 |  97% 
mkl-2019.4           | 204.1 MB  | #########7 |  97% 
mkl-2019.4           | 204.1 MB  | #########7 |  97% 
mkl-2019.4           | 204.1 MB  | #########7 |  97% 
mkl-2019.4           | 204.1 MB  | #########7 |  97% 
mkl-2019.4           | 204.1 MB  | #########7 |  97% 
mkl-2019.4           | 204.1 MB  | #########7 |  97% 
mkl-2019.4           | 204.1 MB  | #########7 |  97% 
mkl-2019.4           | 204.1 MB  | #########7 |  98% 
mkl-2019.4           | 204.1 MB  | #########7 |  98% 
mkl-2019.4           | 204.

Verifying transaction: ...working... done
Executing transaction: ...working... done


  Created wheel for fusepy: filename=fusepy-3.0.1-py3-none-any.whl size=10503 sha256=93dd1864221d8e6c7dd514cba111ed35f27e70a879f891936491e7afb66ed364
  Stored in directory: /root/.cache/pip/wheels/21/5c/83/1dd7e8a232d12227e5410120f4374b33adeb4037473105b079
  Created wheel for smart-open: filename=smart_open-1.9.0-py3-none-any.whl size=73085 sha256=5baa824cd45732d18a25ea0504f47ba4afecb4067ea4fd66eb0277f9e8cd6e42
  Stored in directory: /root/.cache/pip/wheels/8e/9f/cd/dbf5c1362c59abb699a218c1151679033b8ccb5b6db559d512
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78530 sha256=6cb2e79942206eeaa60dc6428423be65bcbbf3202dfb4be512525fda4321598c
  Stored in directory: /root/.cache/pip/wheels/09/84/74/d2b4feb9ac9488bc83c475cb2cbe8e8b7d9cea8320d32f3787
  Created wheel for psutil: filename=psutil-5.7.0-cp36-cp36m-linux_x86_64.whl size=260180 sha256=c1e686a945b011e08f1b4b28fddf9b596939370d57c3b3889d0e6c92a996d3cb
  Stored in directory: /root/.cache/pip/wheels/a1/d9/f2/b5620

...


ephem-3.7.7.0        | 761 KB    |            |   0% 
ephem-3.7.7.0        | 761 KB    | ########8  |  89% 
ephem-3.7.7.0        | 761 KB    | ########## | 100% 

tornado-6.0.4        | 650 KB    |            |   0% 
tornado-6.0.4        | 650 KB    | ########4  |  85% 
tornado-6.0.4        | 650 KB    | ########## | 100% 

python-dateutil-2.8. | 224 KB    |            |   0% 
python-dateutil-2.8. | 224 KB    | ########## | 100% 

mkl_random-1.1.0     | 369 KB    |            |   0% 
mkl_random-1.1.0     | 369 KB    | ########## | 100% 

matplotlib-3.1.3     | 21 KB     |            |   0% 
matplotlib-3.1.3     | 21 KB     | ########## | 100% 

pystan-2.19.0.0      | 16.6 MB   |            |   0% 
pystan-2.19.0.0      | 16.6 MB   | #8         |  18% 
pystan-2.19.0.0      | 16.6 MB   | ####9      |  50% 
pystan-2.19.0.0      | 16.6 MB   | #######5   |  75% 
pystan-2.19.0.0      | 16.6 MB   | #########1 |  91% 

pystan-2.19.0.0      | 16.6 MB   | ########## | 100% 

cython-0.29.15      


mkl-2019.4           | 204.1 MB  | #########5 |  95% 
mkl-2019.4           | 204.1 MB  | #########5 |  95% 
mkl-2019.4           | 204.1 MB  | #########5 |  95% 
mkl-2019.4           | 204.1 MB  | #########5 |  95% 
mkl-2019.4           | 204.1 MB  | #########5 |  96% 
mkl-2019.4           | 204.1 MB  | #########5 |  96% 
mkl-2019.4           | 204.1 MB  | #########5 |  96% 
mkl-2019.4           | 204.1 MB  | #########5 |  96% 
mkl-2019.4           | 204.1 MB  | #########5 |  96% 
mkl-2019.4           | 204.1 MB  | #########5 |  96% 
mkl-2019.4           | 204.1 MB  | #########5 |  96% 
mkl-2019.4           | 204.1 MB  | #########6 |  96% 
mkl-2019.4           | 204.1 MB  | #########6 |  96% 
mkl-2019.4           | 204.1 MB  | #########6 |  96% 
mkl-2019.4           | 204.1 MB  | #########6 |  96% 
mkl-2019.4           | 204.1 MB  | #########6 |  96% 
mkl-2019.4           | 204.1 MB  | #########6 |  96% 
mkl-2019.4           | 204.1 MB  | #########6 |  96% 
mkl-2019.4           | 204.


qt-5.9.6             | 86.7 MB   | #########3 |  93% 
qt-5.9.6             | 86.7 MB   | #########4 |  94% 
qt-5.9.6             | 86.7 MB   | #########4 |  95% 
qt-5.9.6             | 86.7 MB   | #########4 |  95% 
qt-5.9.6             | 86.7 MB   | #########5 |  95% 
qt-5.9.6             | 86.7 MB   | #########5 |  96% 
qt-5.9.6             | 86.7 MB   | #########5 |  96% 
qt-5.9.6             | 86.7 MB   | #########6 |  96% 
qt-5.9.6             | 86.7 MB   | #########6 |  96% 
qt-5.9.6             | 86.7 MB   | #########6 |  96% 
qt-5.9.6             | 86.7 MB   | #########6 |  97% 
qt-5.9.6             | 86.7 MB   | #########6 |  97% 
qt-5.9.6             | 86.7 MB   | #########7 |  97% 
qt-5.9.6             | 86.7 MB   | #########7 |  97% 
qt-5.9.6             | 86.7 MB   | #########7 |  97% 
qt-5.9.6             | 86.7 MB   | #########7 |  98% 
qt-5.9.6             | 86.7 MB   | #########7 |  98% 
qt-5.9.6             | 86.7 MB   | #########8 |  98% 
qt-5.9.6             | 86.7

  Created wheel for fusepy: filename=fusepy-3.0.1-py3-none-any.whl size=10503 sha256=368d988ca3995b1186af75a6eafa59861ff1b800360c3a69b9659efb2cefd3c1
  Stored in directory: /root/.cache/pip/wheels/21/5c/83/1dd7e8a232d12227e5410120f4374b33adeb4037473105b079
  Created wheel for wrapt: filename=wrapt-1.11.1-cp36-cp36m-linux_x86_64.whl size=66711 sha256=8b9678f271932678a71f0dc7f9da858772da8e166cd41428d4e062be4782bce0
  Stored in directory: /root/.cache/pip/wheels/94/0f/ec/66085641573800014bb0c8b657f3366eff641c42df79abbfe9
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78530 sha256=7e2eae01601e6ee97b03c84d199a52c297c6747bd1c3a409c067ef8564e395a7
  Stored in directory: /root/.cache/pip/wheels/09/84/74/d2b4feb9ac9488bc83c475cb2cbe8e8b7d9cea8320d32f3787
  Created wheel for smart-open: filename=smart_open-1.9.0-py3-none-any.whl size=73085 sha256=ab67eacd5e88ef6417a87036216c054c194bdbf815d391b32254b730a2db3fec
  Stored in directory: /root/.cache/pip/wheels/8e/9f/cd/dbf5c13

    Found existing installation: wheel 0.34.2
    Uninstalling wheel-0.34.2:
      Successfully uninstalled wheel-0.34.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


#
# To activate this environment, use:
# > source activate /azureml-envs/azureml_fb099990cc95c5127ccd5b5aee9008eb
#
# To deactivate an active environment, use:
# > source deactivate
#


Removing intermediate container 0933dcfbc791
 ---> 8d2ff2678464
Step 9/15 : ENV PATH /azureml-envs/azureml_fb099990cc95c5127ccd5b5aee9008eb/bin:$PATH
 ---> Running in a36a1b61c4eb
Removing intermediate container a36a1b61c4eb
 ---> 734739ce551f
Step 10/15 : ENV AZUREML_CONDA_ENVIRONMENT_PATH /azureml-envs/azureml_fb099990cc95c5127ccd5b5aee9008eb
 ---> Running in d8a8f75b2d1a
Removing intermediate container d8a8f75b2d1a
 ---> 704f8ada58cb
Step 11/15 : ENV LD_LIBRARY_PATH /azureml-envs/azureml_fb099990cc95c5127ccd5b5aee9008eb/lib:$LD_LIBRAR

The code above combines the data preparation, automated ML, and model-registering steps into a `Pipeline` object. It then creates an `Experiment` object. The `Experiment` constructor will retrieve the named experiment if it exists or create it if necessary. It submits the `Pipeline` to the `Experiment`, creating a `Run` object that will asynchronously run the pipeline. The `wait_for_completion()` function blocks until the run completes.

### Download the results of an automated ML run 

While the `run` object in the code above is from the actively running context, you can also retrieve completed `Run` objects from the `Workspace` by way of an `Experiment` object.

The workspace contains a complete record of all your experiments and runs. You can either use the portal to find and download the outputs of experiments or use code.

In [ ]:
# Run on local machine
experiment = ws.experiments['titanic_automl']
run = next(run for run in ex.get_runs() if run.id == 'aaaaaaaa-bbbb-cccc-dddd-0123456789AB')
automl_run = next(r for r in run.get_children() if r.name == 'AutoML_Classification')
outputs = automl_run.get_outputs()
metrics = outputs['default_metrics_AutoML_Classification']
model = outputs['default_model_AutoML_Classification']

metrics.get_port_data_reference().download('.')
model.get_port_data_reference().download('.')

The above snippet would run on your local machine. First, it logs on to the workspace. It retrieves the `Experiment` named `titanic_automl` and from that `Experiment`, the `Run` in which you're interested. Notice that you'd set the value being compared to `run.id` to that of the run in which you're interested.

Each `Run` object contains `StepRun` objects that contain information about the individual pipeline step run. The `run` is searched for the `StepRun` object for the `AutoMLStep`. The outputs are retrieved using their default names, which are available even if you don't pass `PipelineData` objects to the `outputs` parameter of the `AutoMLStep`. 

Finally, the actual metrics and model are downloaded to your local machine for further processing.


## Next Steps

- Run this Jupyter notebook showing a [complete example of automated ML in a pipeline](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/machine-learning-pipelines/nyc-taxi-data-regression-model-building/nyc-taxi-data-regression-model-building.ipynb) that uses regression to predict taxi fares
- [Create automated ML experiments without writing code](how-to-use-automated-ml-for-ml-models.md)
- Explore a variety of [Jupyter notebooks demonstrating automated ML](https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/automated-machine-learning)
- Read about integrating your pipeline in to [End-to-end MLOps](https://docs.microsoft.com/azure/machine-learning/concept-model-management-and-deployment#automate-the-ml-lifecycle) or investigate the [MLOps Github repository](https://github.com/Microsoft/MLOpspython)